# Raw Layer to Curated Layer (filter and transform using business logic, feature engineering, aggregation)

In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC") # fix timestamps loaded by spark
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/01 20:52:05 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/08/01 20:52:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 20:52:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Yellow Taxis

In [23]:
# make directories for curated data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/curated/'):
    os.makedirs('../data/curated/')
    
if not os.path.exists('../data/curated/tlc'):
    os.makedirs('../data/curated/tlc')
    
if not os.path.exists('../data/curated/tlc/yellow'):
    os.makedirs('../data/curated/tlc/yellow')
    
if not os.path.exists('../data/curated/tlc/yellow/2021'):
    os.makedirs('../data/curated/tlc/yellow/2021')   
    
if not os.path.exists('../data/curated/tlc/yellow/2022'):
    os.makedirs('../data/curated/tlc/yellow/2022')
    
if not os.path.exists('../data/curated/tlc/yellow/2023'):
    os.makedirs('../data/curated/tlc/yellow/2023')

### Load in data

In [3]:
yellow_raw_dir = '../data/raw/tlc/yellow/'

yellow_curated_dir = '../data/curated/tlc/yellow/'

sdf_yellow_2022_01 = spark.read.parquet(f"{yellow_raw_dir}2022/2022-01.parquet")

sdf_yellow_2022_all = spark.read.parquet(f"{yellow_raw_dir}2022/*")

sdf_yellow_2023_01 = spark.read.parquet(f"{yellow_raw_dir}2023/2023-01.parquet")

sdf_yellow_2023_02 = spark.read.parquet(f"{yellow_raw_dir}2023/2023-02.parquet")

#sdf_yellow_2022_all = spark.read.parquet('../data/cleaned/yellow/2022')



In [52]:
sdf_yellow_2022_01.schema

StructType([StructField('vendorid', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('ratecodeid', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [67]:
sdf_yellow_2022_01.where(
    (F.col('airport_fee') == 0)
    & (F.col('pulocationid') != 132)
    & (F.col('pulocationid') != 138)
    | (F.col('pulocationid') == 132)
    & (F.col('airport_fee') == 1.25)
    | (F.col('pulocationid') == 138)
    & (F.col('airport_fee') == 1.25)
    ).where(
        (F.col('airport_fee') != 1.25)
        & (F.col('pulocationid') == 132)
        & (F.col('pulocationid') == 138)
        )

'''
sdf_yellow_2022_01.withColumn(
    'is_valid_record',
    # when we have a positive distance/passenger/total amount then True
    # else False
    F.when(
        (F.col('airport_fee') == 0)
        & (F.col('pulocationid') != 132)
        & (F.col('pulocationid') != 138)
        | (F.col('pulocationid') == 132)
        & (F.col('airport_fee') == 1.25)
        | (F.col('pulocationid') == 138)
        & (F.col('airport_fee') == 1.25),
        True
    ).otherwise(False)
).where(F.col('airport_fee') == 1.25)
'''
sdf_yellow_2022_01.where(
    (F.col('vendorid') == 1)
    | (F.col('vendorid') == 2)
    ).where(
        (F.col('vendorid') == 1)
        & (F.col('vendorid') != 2)
)
    
sdf_yellow_2022_01.where(
    (F.col('extra') == 0.50) # ensure extra charges only applied for rush hour and overnight trips
    & (F.hour('tpep_pickup_datetime') >= 20)
    & (F.hour('tpep_pickup_datetime') < 6)
    | (F.col('extra') == 0.50) #
    & (F.hour('tpep_dropoff_datetime') >= 20)
    & (F.hour('tpep_dropoff_datetime') < 6)
    | (F.col('extra') == 1.00)
    & (F.hour('tpep_pickup_datetime') >= 16)
    & (F.hour('tpep_pickup_datetime') < 20)
    | (F.col('extra') == 1.00) 
    & (F.hour('tpep_dropoff_datetime') >= 16)
    & (F.hour('tpep_dropoff_datetime') < 20)
    | (F.col('extra') == 0) 
    & (F.hour('tpep_pickup_datetime') >= 6)
    & (F.hour('tpep_pickup_datetime') < 16)
    & (F.hour('tpep_dropoff_datetime') >= 6)
    & (F.hour('tpep_dropoff_datetime') < 16)
).where(
    (F.col('extra') != 0)
    & (F.col('extra') != 1.0)
)





sdf_yellow_2022_01.where(
        (F.col('extra') == 1.00) # ensure rush hour surcharge only applied for trips during rush hour
        & (F.hour('tpep_pickup_datetime') >= 16)
        & (F.hour('tpep_pickup_datetime') < 20)
        | (F.col('extra') == 1.00) 
        & (F.hour('tpep_dropoff_datetime') >= 16)
        & (F.hour('tpep_dropoff_datetime') < 20)
        | (F.col('extra') == 0) 
        & (F.hour('tpep_pickup_datetime') >= 6)
        & (F.hour('tpep_pickup_datetime') < 16)
        & (F.hour('tpep_dropoff_datetime') >= 6)
        & (F.hour('tpep_dropoff_datetime') < 16)
        | (F.col('extra') == 0.50)
        & (F.hour('tpep_pickup_datetime') >= 20)
        | (F.col('extra') == 0.50) 
        & (F.hour('tpep_pickup_datetime') >= 20)
        | (F.hour('tpep_pickup_datetime') < 6)
        | (F.col('extra') == 0.50)
        & (F.hour('tpep_pickup_datetime') >= 20)
        | (F.hour('tpep_dropoff_datetime') < 6)
    ).where(
        (F.col('extra') == 0.5)
        & (F.hour('tpep_pickup_datetime') >= 20)
        & (F.dayofmonth('tpep_pickup_datetime') == 2)
    )
    
print(sdf_yellow_2022_01.dropna('any').count())

print(sdf_yellow_2022_01.count())

2392428
2463931


In [30]:
taxi_zones = spark.read.parquet('../data/raw/tlc/taxi_zones/taxi_zone_lookup.parquet/')

jfk_id = int(taxi_zones.where(
    F.col('zone') == 'JFK Airport'
).select('locationid').first()['locationid'])

laguardia_id = int(taxi_zones.where(
    F.col('zone') == 'LaGuardia Airport'
).select('locationid').first()['locationid'])


138

### Use business logic to filter and transform data

In [61]:
payment_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']

from pyspark.sql import functions as F

# 2022
for month in range(1,12):

    year = 2022
    sdf = spark.read.parquet(f"{yellow_raw_dir}2022/2022-{str(month).zfill(2)}.parquet")  

    #laguardia_id = 
    #jfk_id = 
    
    '''
    sdf = sdf.withColumn(
        'is_valid_record',
        # when we have a positive distance/passenger/total amount then True
        # else False
        F.when(
            (F.col('trip_distance') > 0)
            & (F.col('passenger_count') > 0)
            & (F.col('total_amount') > 0),
            True
        ).otherwise(False)
    )
    '''
    
    # may have to go through data and apply invalid record boolean attribute first for more complex business logic checking such as airport fee matching pickup at laguardia / jfk
    
    # remove invalid records by applying business logic
    sdf = \
    sdf.where(
        (F.year(F.col('tpep_pickup_datetime')) == year) # ensure that this trip was initiated within the correct year (assuming a trip belongs to a year based on pickup time)
        (F.month(F.col('tpep_pickup_datetime')) == month)  # ensure that this trip was initiated within the correct month (assuming a trip belongs to a month based on pickup time)
        & (F.unix_timestamp('tpep_pickup_datetime') < F.unix_timestamp('tpep_dropoff_datetime')) # make sure pickup time is earlier than dropoff time
        & (F.col('passenger_count') > 0) # ensure non-zero passenger count, but also within 6 people based on the law
        & (F.col('passenger_count') <= 6)
        & (F.col('trip_distance') > 0.25) # remove extremely short / negative trip distance based on distance people are willing to walk before taking a taxi
        & (F.col('trip_distance') < 300) # remove extreme outliers for trip distance (trips around and above this distance generally appear invalid / erroneous)
        & (F.col('pulocationid') >= 1) # ensure trips are only within the specified range of locations
        & (F.col('pulocationid') <= 263)
        & (F.col('dolocationid') >= 1)
        & (F.col('dolocationid') <= 263)
        & (F.col('total_amount') >= 2.50) # ensure non-negative fees, minimum of $2.50 for standard fares
        & (F.col('fare_amount') >= 2.50) 
        & (F.col('tip_amount') >= 0) 
        & (F.col('tolls_amount') >= 0)
        & (F.col('payment_type') == 1) # include only credit card payments as cash tips are not included in this data
        & (F.col('total_amount') == sum(sdf[col] for col in payment_cols)) # check the total amount equals the sum of all other fees
    ).where(
        (F.col('vendorid') == 1) # check for valid vendor id
        | (F.col('vendorid') == 2)  
    )
    .where(
        (F.col('ratecodeid') == 1) # standard, JFK, and Newark rates suitable for this research based on the locations covered
        | (F.col('ratecodeid') == 2)  
        | (F.col('ratecodeid') == 3)
    ).where(
        (F.col('mta_tax') == 0.50) # $0.50 MTA tax for trips that end in NYC (but not Newark).
        & (F.col('dolocationid') != 1)  
        | (F.col('mta_tax') == 0)
        & (F.col('dolocationid') == 1)  
    ).where(
        (F.col('improvement_surcharge') == 0) # check improvement surcharge is either $0 or $0.30
        | (F.col('improvement_surcharge') == 0.30)
    ).where(
        (F.col('congestion_surcharge') == 0) # check congestion surcharge is either $0 or $2.50 for yellow taxis
        | (F.col('congestion_surcharge') == 2.50)
    ).where(
        (F.col('extra') == 1.00) # ensure rush hour surcharge only applied for trips during rush hour, 
        & (F.hour('tpep_pickup_datetime') >= 16)
        & (F.hour('tpep_pickup_datetime') < 20)
        | (F.col('extra') == 1.00) 
        & (F.hour('tpep_dropoff_datetime') >= 16)
        & (F.hour('tpep_dropoff_datetime') < 20)
        | (F.col('extra') == 0)  # no surcharge for trips that don't fall within rush hour or overnight time slots
        & (F.hour('tpep_pickup_datetime') >= 6)
        & (F.hour('tpep_pickup_datetime') < 16)
        & (F.hour('tpep_dropoff_datetime') >= 6)
        & (F.hour('tpep_dropoff_datetime') < 16)
        | (F.col('extra') == 0.50) # ensure overnight surcharge only applied for overnight trips (rush hour surcharge takes precedent over overnight surcharge when a trip falls within both time slots)
        & (F.hour('tpep_pickup_datetime') >= 20) 
        & (F.hour('tpep_pickup_datetime') < 6)
        | (F.col('extra') == 0.50)
        & (F.hour('tpep_pickup_datetime') >= 20)
        | (F.hour('tpep_dropoff_datetime') < 6)
    ).where(
        (F.col('airport_fee') == 0) # ensure airport fee was only applied for pickup at JFK or LaGuardia
        & (F.col('pulocationid') != jfk_id)
        & (F.col('pulocationid') != laguardia_id)
        | (F.col('pulocationid') == jfk_id)
        & (F.col('airport_fee') == 1.25)
        | (F.col('pulocationid') == laguardia_id)
        & (F.col('airport_fee') == 1.25)  
    ).drop( 
        'store_and_fwd_flag', 'total_amount' # remove store_and_fwd flag (as this is useless for our analysis) and total_amount (as this can be calculated from the other features)
    ).dropna('any') # remove any records containing NULL values as it will be assumed these are invalid / erroneous entries and PySpark ML is unable to handle NULL values
    
    '''
    sdf_cleaned = sdf_cleaned.withColumn(
        'store_and_fwd_flag',
        (F.col("store_and_fwd_flag") == 'Y').cast('BOOLEAN')
    )
    '''
        
    sdf \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{yellow_curated_dir}2022/2022-{str(month).zfill(2)}.parquet")

'''
# 2023
for month in range(1,5):

    year = 2023
    sdf_cleaned = spark.read.parquet(f"{yellow_raw_dir}2023/2023-{str(month).zfill(2)}.parquet")
    
    # apply business logic
    
   
    
    
    
    
    
       
    sdf_cleaned \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{yellow_curated_dir}2023/2023-{str(month).zfill(2)}.parquet")
'''    
    
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

TypeError: 'Column' object is not callable

In [ ]:
sdf.show(1, vertical=True, truncate=100)

### Feature Engineering

### Aggregation

In [ ]:
# double check curated 2022 data

sdf1 = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/yellow/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

In [ ]:
# double check curated 2023 data

sdf = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/yellow/2023/*')

sdf.show(1, vertical=True, truncate=100)

## Green Taxis

In [25]:
# make directories for curated data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/curated/'):
    os.makedirs('../data/curated/')
    
if not os.path.exists('../data/curated/tlc'):
    os.makedirs('../data/curated/tlc')
    
if not os.path.exists('../data/curated/tlc/green'):
    os.makedirs('../data/curated/tlc/green')
    
if not os.path.exists('../data/curated/tlc/green/2021'):
    os.makedirs('../data/curated/tlc/green/2021')   
    
if not os.path.exists('../data/curated/tlc/green/2022'):
    os.makedirs('../data/curated/tlc/green/2022')
    
if not os.path.exists('../data/curated/tlc/green/2023'):
    os.makedirs('../data/curated/tlc/green/2023')

In [20]:
green_raw_dir = '../data/raw/tlc/green/'

sdf_green_2022_01 = spark.read.parquet(f"{green_raw_dir}2022/2022-01.parquet")

sdf_green_2022_all = spark.read.parquet(f"{green_raw_dir}2022/*")

sdf_green_2023_01 = spark.read.parquet(f"{green_raw_dir}2023/2023-01.parquet")

sdf_green_2023_02 = spark.read.parquet(f"{green_raw_dir}2023/2023-02.parquet")

#sdf_green_2022_all = spark.read.parquet('../data/raw/green/2022')



In [21]:
sdf_green_2022_01.schema

StructType([StructField('vendorid', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [47]:
sdf_green_2022_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [48]:
sdf_green_2023_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [49]:
sdf_green_2023_02.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', Doubl

In [68]:
# Test filters here
#sdf_green_2022_01.schema

payment_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge']

print("Total trips: ")

print(sdf_green_2022_01.count())

print("Credit card payment trips with valid total: ")

print(sdf_green_2022_01.where((F.col('payment_type') == 1) & (F.col('total_amount') == sum(sdf_green_2022_01[col] for col in payment_cols))).count())

print("Trips with valid total: ")

print(sdf_green_2022_01.where(F.col('total_amount') == sum(sdf_green_2022_01[col] for col in payment_cols)).count())

print("Cash payment trips with valid total: ")

print(sdf_green_2022_01.where((F.col('payment_type') == 2) & (F.col('total_amount') == sum(sdf_green_2022_01[col] for col in payment_cols))).count())

sdf_green_2022_01.where(
    (F.col('congestion_surcharge') != 0) 
)

#sdf_green_2022_01.filter(F.col('passenger_count') == 5).limit(5)

Total trips: 
2463931
Credit card payment trips with valid total: 
1039104
Trips with valid total: 
1399649
Cash payment trips with valid total: 
348632


NameError: name 'sdf_green_2022_01' is not defined

In [19]:
# obtain a list of 'yellow zones' (zones in which only yellow taxis may respond to street hails)

taxi_zones = spark.read.parquet('../data/raw/tlc/taxi_zones/taxi_zone_lookup.parquet/')

yellow_zones = taxi_zones.where(
    F.col('service_zone') == 'Yellow Zone'
).select('locationid').rdd.map(lambda x: x.locationid).collect()

yellow_zones = list(map(int, yellow_zones))

sdf_green_2022_01.where(
    (~F.col('pulocationid').isin(yellow_zones)) # only keep street hail entries that aren't recorded as pickup in yellow zones (as green taxis can't respond to street hails in these zones)
        | (F.col('pulocationid').isin(yellow_zones))
        & (F.col('trip_type') == 2)
).where(
    (F.col('pulocationid').isin(yellow_zones))
    & (F.col('trip_type') == 1)
)



vendorid,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecodeid,pulocationid,dolocationid,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge


### Use business logic to filter and transform data

In [ ]:
payment_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge']

from pyspark.sql import functions as F

green_raw_dir = '../data/raw/tlc/green/'
    
green_curated_dir = '../data/curated/tlc/green/'

YEARS = ['2021', '2022', '2023']

for year in YEARS:
    if (year == '2021'):
        MONTHS = [12]
    if (year == '2022'):
        MONTHS = range(1,13)
    if (year == '2023'):   
        MONTHS = range(1,5) 
    for month in MONTHS:
        
        sdf = spark.read.parquet(f"{green_raw_dir}{year}/{year}-{str(month).zfill(2)}.parquet")  


        
        '''
        sdf = sdf.withColumn(
            'is_valid_record',
            # when we have a positive distance/passenger/total amount then True
            # else False
            F.when(
                (F.col('trip_distance') > 0)
                & (F.col('passenger_count') > 0)
                & (F.col('total_amount') > 0),
                True
            ).otherwise(False)
        )
        '''
        
        # may have to go through data and apply invalid record boolean attribute first for more complex business logic checking such as airport fee matching pickup at laguardia / jfk
        
        # remove invalid records by applying business logic
        sdf = \
        sdf.where(
            (F.year(F.col('tpep_pickup_datetime')) == year) # ensure that this trip was initiated within the correct year (assuming a trip belongs to a year based on pickup time)
            (F.month(F.col('tpep_pickup_datetime')) == month)  # ensure that this trip was initiated within the correct month (assuming a trip belongs to a month based on pickup time)
            & (F.unix_timestamp('tpep_pickup_datetime') < F.unix_timestamp('tpep_dropoff_datetime')) # make sure pickup time is earlier than dropoff time
            & (F.col('passenger_count') > 0) # ensure non-zero passenger count, but also within 6 people based on the law
            & (F.col('passenger_count') <= 6)
            & (F.col('trip_distance') > 0.25) # remove extremely short / negative trip distance based on distance people are willing to walk before taking a taxi
            & (F.col('trip_distance') < 300) # remove extreme outliers for trip distance (trips around and above this distance generally appear invalid / erroneous)
            & (F.col('pulocationid') >= 1) # ensure trips are only within the specified range of locations
            & (F.col('pulocationid') <= 263)
            & (F.col('dolocationid') >= 1)
            & (F.col('dolocationid') <= 263)
            & (F.col('total_amount') >= 2.50) # ensure non-negative fees, minimum of $2.50 for standard fares
            & (F.col('fare_amount') >= 2.50) 
            & (F.col('tip_amount') >= 0) 
            & (F.col('tolls_amount') >= 0)
            & (F.col('payment_type') == 1) # include only credit card payments as cash tips are not included in this data
            & (F.col('total_amount') == sum(sdf[col] for col in payment_cols)) # check the total amount equals the sum of all other fees
        ).where(
            (F.col('vendorid') == 1) # check for valid vendor id
            | (F.col('vendorid') == 2)  
        )
        .where(
            (F.col('ratecodeid') == 1) # standard, JFK, and Newark rates suitable for this research based on the locations covered
            | (F.col('ratecodeid') == 2)  
            | (F.col('ratecodeid') == 3)
        ).where(
            (F.col('mta_tax') == 0.50) # $0.50 MTA tax for trips that end in NYC (but not Newark according to taxi fare page).
            & (F.col('dolocationid') != 1)  
            | (F.col('mta_tax') == 0)
            & (F.col('dolocationid') == 1)  
        ).where(
            (F.col('improvement_surcharge') == 0) # check improvement surcharge is either $0 or $0.30
            | (F.col('improvement_surcharge') == 0.30)
        ).where(
            (F.col('congestion_surcharge') == 0) # check congestion surcharge is either $0 or $2.75 for green taxis
            | (F.col('congestion_surcharge') == 2.75)
        ).where(
            (F.col('extra') == 1.00) # ensure rush hour surcharge only applied for trips during rush hour, 
            & (F.hour('tpep_pickup_datetime') >= 16)
            & (F.hour('tpep_pickup_datetime') < 20)
            | (F.col('extra') == 1.00) 
            & (F.hour('tpep_dropoff_datetime') >= 16)
            & (F.hour('tpep_dropoff_datetime') < 20)
            | (F.col('extra') == 0)  # no surcharge for trips that don't fall within rush hour or overnight time slots
            & (F.hour('tpep_pickup_datetime') >= 6)
            & (F.hour('tpep_pickup_datetime') < 16)
            & (F.hour('tpep_dropoff_datetime') >= 6)
            & (F.hour('tpep_dropoff_datetime') < 16)
            | (F.col('extra') == 0.50) # ensure overnight surcharge only applied for overnight trips (rush hour surcharge takes precedent over overnight surcharge when a trip falls within both time slots)
            & (F.hour('tpep_pickup_datetime') >= 20) 
            & (F.hour('tpep_pickup_datetime') < 6)
            | (F.col('extra') == 0.50)
            & (F.hour('tpep_pickup_datetime') >= 20)
            | (F.hour('tpep_dropoff_datetime') < 6)
        )where(
            (~F.col('pulocationid').isin(yellow_zones)) # only keep street hail entries (trip_type == 1) that aren't recorded as pickup in yellow zones 
            | (F.col('pulocationid').isin(yellow_zones)) # (as green taxis can't respond to street hails in these zones)
            & (F.col('trip_type') == 2)
        ).drop( 
            'store_and_fwd_flag', 'total_amount' # remove store_and_fwd flag (as this is useless for our analysis) and total_amount (as this can be calculated from the other features)
        ).dropna('any') # remove any records containing NULL values as it will be assumed these are invalid / erroneous entries and PySpark ML is unable to handle NULL values
        
        '''
        sdf_cleaned = sdf_cleaned.withColumn(
            'store_and_fwd_flag',
            (F.col("store_and_fwd_flag") == 'Y').cast('BOOLEAN')
        )
        '''
            
        sdf \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet(f"{green_curated_dir}2022/2022-{str(month).zfill(2)}.parquet")

    
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

TypeError: 'Column' object is not callable

In [ ]:
sdf.show(1, vertical=True, truncate=100)

### Feature Engineering

### Aggregation

In [ ]:
# double check curated 2022 data

sdf1 = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/yellow/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

In [67]:
# double check schema of 2022 cleaned data

sdf1 = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/green/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 lpep_pickup_datetime  | 2022-03-01 00:24:14 
 lpep_dropoff_datetime | 2022-03-01 00:34:04 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 74                  
 dolocationid          | 151                 
 passenger_count       | 1                   
 trip_distance         | 2.3                 
 fare_amount           | 9.5                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 0.3                 
 total_amount          | 10.8                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [68]:
# double check schema of 2023 cleaned data

sdf = spark.read.schema(sdf_schema).parquet('../data/cleaned/tlc/green/2023/*')

sdf.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 vendorid              | 2                   
 lpep_pickup_datetime  | 2023-03-01 00:25:10 
 lpep_dropoff_datetime | 2023-03-01 00:35:47 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 82                  
 dolocationid          | 196                 
 passenger_count       | 1                   
 trip_distance         | 2.36                
 fare_amount           | 13.5                
 extra                 | 1.0                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 1.0                 
 total_amount          | 16.0                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



In [75]:
# make directories for cleaned data

import os

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/taxi_zones'):
    os.makedirs('../data/cleaned/tlc/taxi_zones')

In [5]:
taxi_zone_lookup = spark.read.option("header",True) \
                   .csv("../data/raw/tlc/taxi_zones/taxi+_zone_lookup.csv")

In [10]:
print(taxi_zone_lookup.limit(5))

print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])


### Just need to convert the column names to lowercase for this data

In [12]:
from pyspark.sql import functions as F

# converting to lower case
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in taxi_zone_lookup.columns]
taxi_zone_lookup = taxi_zone_lookup.select(*consistent_col_casing)

print(taxi_zone_lookup.limit(5))
print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('locationid', StringType(), True), StructField('borough', StringType(), True), StructField('zone', StringType(), True), StructField('service_zone', StringType(), True)])


In [13]:
# save cleaned data as parquet
taxi_zone_lookup \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/cleaned/tlc/taxi_zones/taxi_zone_lookup.parquet')

In [14]:
# check cleaned data
taxi_zone_lookup = spark.read.parquet('../data/cleaned/tlc/taxi_zones/taxi_zone_lookup.parquet')

taxi_zone_lookup.printSchema


<bound method DataFrame.printSchema of +----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City

## External Datasets

### Motor Vehicle Collisions

### Subway Entrances and Exits

### Subway Hourly Ridership

### Bus Hourly Ridership

### Hotel Properties

### Airbnb

### Census Data